In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display # Allows the use of display() for DataFrames

# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

import time
import datetime
import os

#import datacleaningp1 as dcp
import dcp
reload(dcp)
date_list = dcp.datelist((2016, 1, 1), (2016, 1, 22))  ### generate time list of training data
print date_list

['2016-01-01', '2016-01-02', '2016-01-03', '2016-01-04', '2016-01-05', '2016-01-06', '2016-01-07', '2016-01-08', '2016-01-09', '2016-01-10', '2016-01-11', '2016-01-12', '2016-01-13', '2016-01-14', '2016-01-15', '2016-01-16', '2016-01-17', '2016-01-18', '2016-01-19', '2016-01-20', '2016-01-21']


In [2]:
# load training data
data_train_all = pd.read_csv('TrainingData_fixed_4_test_1.csv')
print 'Training data read sucessfully'
display(data_train_all.describe())

Training data read sucessfully


,district_id,time,gaps,gaps_pre_1,gaps_pre_2,gaps_pre_3,PM2.5,Weather_1,Weather_2,Weather_3,Weather_4,Weather_6,Weather_8,Weather_9,temperature,tj_level_1,tj_level_2,tj_level_3,tj_level_4
count,194297.000000,194297.000000,194297.000000,194297.000000,194297.000000,194297.000000,194297.000000,194297.000000,194297.00000,194297.000000,194297.000000,194297.0,194297.000000,194297.000000,194297.000000,194297.000000,194297.000000,194297.000000,194297.000000
mean,28.930766,77.592366,14.280442,13.779158,13.655780,13.579541,114.856112,0.015440,0.47852,0.163106,0.271306,0.0,0.070444,0.001184,6.444562,922.872895,211.266116,63.811501,42.607770
std,17.260492,38.605024,57.923842,58.017603,58.020468,58.012699,47.721755,0.123296,0.49954,0.369463,0.444635,0.0,0.255894,0.034385,3.812221,821.765282,271.129976,85.259852,51.901978
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,26.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,14.000000,50.000000,1.000000,0.000000,0.000000,0.000000,81.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,3.000000,292.000000,44.000000,13.000000,8.000000
50%,27.000000,78.000000,3.000000,2.000000,2.000000,2.000000,110.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,6.000000,678.000000,119.000000,35.000000,25.000000
75%,42.000000,110.000000,7.000000,7.000000,6.000000,6.000000,148.000000,0.000000,1.00000,0.000000,1.000000,0.0,0.000000,0.000000,8.000000,1357.000000,288.000000,83.000000,58.000000
max,66.000000,144.000000,3867.000000,3867.000000,3867.000000,3867.000000,241.000000,1.000000,1.00000,1.000000,1.000000,0.0,1.000000,1.000000,19.000000,4223.000000,2084.000000,863.000000,486.000000


## Test1: train a knn-model for district 1

In [3]:
data_1 = data_train_all[data_train_all['district_id'] == 1]
data_1.drop(['district_id'],inplace = True, axis = 1)
data_1 = data_1.reset_index(drop = True)
display(data_1.describe())

C:\Users\sjtu\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,time,gaps,gaps_pre_1,gaps_pre_2,gaps_pre_3,PM2.5,Weather_1,Weather_2,Weather_3,Weather_4,Weather_6,Weather_8,Weather_9,temperature,tj_level_1,tj_level_2,tj_level_3,tj_level_4
count,5205.000000,5205.00000,5205.000000,5205.000000,5205.000000,5205.000000,5205.000000,5205.000000,5205.000000,5205.000000,5205.0,5205.000000,5205.000000,5205.000000,5205.000000,5205.000000,5205.000000,5205.000000
mean,76.586167,6.46609,6.247839,6.191547,6.121037,114.995389,0.018828,0.467243,0.165802,0.280115,0.0,0.066859,0.001153,6.421326,1344.725072,298.036503,94.947166,76.108549
std,40.573001,14.81602,14.877700,14.885587,14.608783,47.320261,0.135931,0.498974,0.371939,0.449098,0.0,0.249801,0.033936,3.756855,256.057480,110.754374,38.470315,26.230024
min,1.000000,1.00000,0.000000,0.000000,0.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,552.000000,57.000000,12.000000,12.000000
25%,45.000000,2.00000,2.000000,2.000000,2.000000,81.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,3.000000,1280.000000,223.000000,69.000000,57.000000
50%,78.000000,4.00000,4.000000,4.000000,4.000000,111.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,6.000000,1445.000000,316.000000,97.000000,78.000000
75%,111.000000,7.00000,7.000000,7.000000,7.000000,148.000000,0.000000,1.000000,0.000000,1.000000,0.0,0.000000,0.000000,8.000000,1497.000000,374.000000,120.000000,94.000000
max,144.000000,301.00000,301.000000,301.000000,301.000000,241.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,19.000000,1861.000000,638.000000,223.000000,162.000000


In [4]:
### Shuffle data, then do train_test_split
from sklearn.utils import shuffle
data_1_s = shuffle(data_1,random_state = 42)

display(data_1_s.head(10))

,time,gaps,gaps_pre_1,gaps_pre_2,gaps_pre_3,PM2.5,Weather_1,Weather_2,Weather_3,Weather_4,Weather_6,Weather_8,Weather_9,temperature,tj_level_1,tj_level_2,tj_level_3,tj_level_4
468,77.0,5.0,4.0,4.0,3.0,167.0,0.0,1.0,0.0,0.0,0,0.0,0.0,17.0,1485.0,299.0,92.0,85.0
296,24.0,10.0,2.0,1.0,1.0,179.0,0.0,1.0,0.0,0.0,0,0.0,0.0,8.0,628.0,87.0,37.0,34.0
4613,73.0,5.0,0.0,6.0,6.0,148.0,0.0,1.0,0.0,0.0,0,0.0,0.0,0.0,1514.0,288.0,78.0,96.0
4092,68.0,1.0,1.0,2.0,5.0,104.0,0.0,0.0,0.0,0.0,0,1.0,0.0,6.0,1426.0,298.0,124.0,89.0
3532,35.0,2.0,2.0,2.0,0.0,161.0,0.0,0.0,1.0,0.0,0,0.0,0.0,4.0,806.0,135.0,46.0,42.0
4375,88.0,3.0,5.0,6.0,3.0,81.0,0.0,0.0,1.0,0.0,0,0.0,0.0,3.0,1430.0,243.0,79.0,72.0
132,134.0,7.0,5.0,2.0,10.0,128.0,0.0,1.0,0.0,0.0,0,0.0,0.0,8.0,1400.0,304.0,117.0,73.0
848,73.0,2.0,4.0,6.0,3.0,219.0,0.0,0.0,0.0,1.0,0,0.0,0.0,12.0,1441.0,338.0,113.0,101.0
1183,134.0,7.0,5.0,1.0,5.0,48.0,0.0,0.0,0.0,1.0,0,0.0,0.0,8.0,1396.0,332.0,95.0,60.0
1117,68.0,1.0,1.0,0.0,5.0,26.0,0.0,0.0,0.0,1.0,0,0.0,0.0,8.0,1469.0,333.0,99.0,77.0


In [5]:
### split labels from data set
label = data_1_s['gaps']
data_1_s.drop(['gaps'],axis = 1,inplace = True)

### feature normalization
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
data_1_n = preprocessing.MinMaxScaler().fit_transform(data_1_s)

X_train, X_test, y_train, y_test = train_test_split(data_1_n, label, test_size=0.3, random_state=42)
print 'Training set size: ',len(X_train)
print 'Test set size: ',len(X_test)

C:\Users\sjtu\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


Training set size:  3643
Test set size:  1562


In [6]:
from sklearn.neighbors import KNeighborsRegressor

neigh = KNeighborsRegressor(n_neighbors= 1)
neigh.fit(X_train,y_train)

y_pred = neigh.predict(X_test)
from sklearn.metrics import mean_absolute_error,r2_score

print 'r2 score on test set %d is '%1, r2_score(y_test,y_pred)
print 'r2 score on training set is ',neigh.score(X_train,y_train)
print 'MAE is ',mean_absolute_error(y_test,y_pred)

r2 score on test set 1 is  0.530569494134
r2 score on training set is  1.0
MAE is  2.17861715749


Use k = 1, a simple Nearest Neighbor Regressor

## Test 2 take average on all 66 district

In [13]:
district_number = 66
test_set_size = 0.3
num_neighbors = 1

MAE_list = []
for iid in np.arange(district_number)+1:
    ###select training data for one specific district
    data_iid = data_train_all[data_train_all['district_id'] == iid]
    data_iid.drop(['district_id'],inplace = True, axis = 1)
    data_iid = data_iid.reset_index(drop = True)
    
    ### shuffle data
    data_iid_s = shuffle(data_iid,random_state = 42)
    ### split labels from data set
    label = data_iid_s['gaps']
    data_iid_s.drop(['gaps'],axis = 1,inplace = True)
    
    ### feature renormalization & train_test_split
    data_iid_n = preprocessing.MinMaxScaler().fit_transform(data_iid_s)
    X_train, X_test, y_train, y_test = train_test_split(data_iid_n, label, test_size= test_set_size, random_state=42)
    
    ### train a nearest neighbor regressor
    neigh = KNeighborsRegressor(n_neighbors= num_neighbors)
    neigh.fit(X_train,y_train)
    
    y_pred = neigh.predict(X_test)
    MAE_iid = mean_absolute_error(y_test,y_pred)
    print 'r2 score on test set %d is '%iid, r2_score(y_test,y_pred)
    print 'MAE is ',mean_absolute_error(y_test,y_pred)
    print '\n'
    
    MAE_list.append(MAE_iid)

C:\Users\sjtu\Anaconda2\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\sjtu\Anaconda2\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


r2 score on test set 1 is  0.530569494134
MAE is  2.17861715749


r2 score on test set 2 is  0.377030162413
MAE is  0.34


r2 score on test set 3 is  0.308939829
MAE is  0.451851851852


r2 score on test set 4 is  0.880455393572
MAE is  1.98255813953


r2 score on test set 5 is  0.519656414403
MAE is  0.512333965844


r2 score on test set 6 is  0.763832091904
MAE is  1.59939455096


r2 score on test set 7 is  0.845773092717
MAE is  6.13230950975


r2 score on test set 8 is  0.896187267959
MAE is  9.25586995786


r2 score on test set 9 is  0.751327169659
MAE is  0.956260720412


r2 score on test set 10 is  0.373664722912
MAE is  0.767710049423


r2 score on test set 11 is  0.429041687914
MAE is  0.449561403509


r2 score on test set 12 is  0.750801021188
MAE is  1.19534282018


r2 score on test set 13 is  0.165224528883
MAE is  0.442477876106


r2 score on test set 14 is  0.90351455202
MAE is  2.80559254328


r2 score on test set 15 is  0.570285003356
MAE is  0.411428571429


r2 score o

In [14]:
MAE_all = np.array(MAE_list)
print MAE_all.mean()

1.72563849122
